In [1]:
import pandas as pd
import pickle
import gzip
import json

In [2]:
#Code adapted from the datasource http://jmcauley.ucsd.edu/data/amazon/
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [3]:
#Convert json to pd dataframe
df = getDF('Home_and_Kitchen_5.json.gz')

In [4]:
#Check datatype and information
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6898955 entries, 0 to 6898954
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   overall         float64
 1   verified        bool   
 2   reviewTime      object 
 3   reviewerID      object 
 4   asin            object 
 5   reviewerName    object 
 6   reviewText      object 
 7   summary         object 
 8   unixReviewTime  int64  
 9   vote            object 
 10  style           object 
 11  image           object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 638.2+ MB


In [5]:
#Visualize the data
df

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,True,"11 5, 2015",A8LUWTIPU9CZB,0560467893,Linda Fahner,"Great product, love it!!",Five Stars,1446681600,NaN,NaN,NaN
1,3.0,True,"05 7, 2015",A3B6GKQQ1JJ167,0560467893,Harry Slaughter,"Pretty flimsy, but does the job. If your corne...",Meh,1430956800,2,NaN,NaN
2,5.0,True,"01 22, 2014",A3MCTN65BU7XRA,0681795107,luckyg,So much better than plastic mug types--keeps c...,Recommend,1390348800,NaN,{'Color:': ' Brushed Stainless'},NaN
3,1.0,True,"10 30, 2013",A7JVZFSXVY9RL,0681795107,Nickleen,I like my coffee hot; borderline scorching but...,Not keeping coffee hot for long enough,1383091200,NaN,{'Color:': ' Brushed Stainless'},NaN
4,1.0,True,"09 20, 2013",A2RQ7VLAK1SHPU,0681795107,Lacemaker427,This mug does only a fair job of keeping coffe...,Leaks like a waterfall when at an angle!,1379635200,NaN,{'Color:': ' Red'},NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6898950,5.0,True,"08 8, 2016",AB4CZUDHN52H5,B01HJEJDBQ,J. Mayer,I purchased these as decorations for my classr...,Beautiful Pom poms,1470614400,NaN,{'Color:': ' Mint Green/Purple/White'},NaN
6898951,2.0,True,"09 5, 2017",A12NA9GEGY6I3E,B01HJEOT2E,Loreli,getting the used tea leaves out of this is ext...,cute but not very functional,1504569600,NaN,{'Color:': ' Blue'},NaN
6898952,4.0,True,"07 30, 2017",A38PQCNDGGWSPQ,B01HJEOT2E,Becca,Good deal. It took a while to recieve so I kin...,Good deal. It took a while to recieve so ...,1501372800,NaN,{'Color:': ' Blue'},NaN
6898953,4.0,True,"07 28, 2018",A2AY2C5EW8VOO7,B01HJGJNWS,Dr. Quinzel,Great price and okay quality. I know it's not ...,Four Stars,1532736000,NaN,{'Color:': ' Black'},NaN


## Data Preparation and Cleaning
#### 1. The column name 'vote','style','image' does not provide much information for this project and 'reviewerName' is not required since the dataset contains 'reviewer id'
#### 2. Drop all the unverified reviews
#### 3. Conver reviewtime to date format 
#### 4. Find all duplicate entries (I.e If the reviewer has reviewed the same product more than once, Keep the latest review)
#### 5. Check for null values remove entries where both summary and reviewtext are nan
#### 6. Find all the url in summary text and replace them with ''
#### 7. Combine Review text and summary column and drop summary column and reset index
#### 8. Find all the url in review text and remove
#### 9. Check for null values
#### 10. Save pickle file

In [6]:
#Drop columns that are not required for analysis
df.drop(columns=['vote','style','image','reviewerName'],inplace=True)

In [7]:
#View top 5 rows
df.head(5)

,overall,verified,reviewTime,reviewerID,asin,reviewText,summary,unixReviewTime
0,5.0,True,"11 5, 2015",A8LUWTIPU9CZB,0560467893,"Great product, love it!!",Five Stars,1446681600
1,3.0,True,"05 7, 2015",A3B6GKQQ1JJ167,0560467893,"Pretty flimsy, but does the job. If your corne...",Meh,1430956800
2,5.0,True,"01 22, 2014",A3MCTN65BU7XRA,0681795107,So much better than plastic mug types--keeps c...,Recommend,1390348800
3,1.0,True,"10 30, 2013",A7JVZFSXVY9RL,0681795107,I like my coffee hot; borderline scorching but...,Not keeping coffee hot for long enough,1383091200
4,1.0,True,"09 20, 2013",A2RQ7VLAK1SHPU,0681795107,This mug does only a fair job of keeping coffe...,Leaks like a waterfall when at an angle!,1379635200


In [8]:
#Check for the unique values in verified column
df.verified.unique()

array([ True, False])

In [9]:
#Drop all the unverified reviews
drp_ind = df[df['verified']==False].index
df.drop(drp_ind,inplace=True)

In [10]:
#Covert unixReview time to date date format and add a column 'rev_date'
df['rev_date'] = pd.to_datetime(df.unixReviewTime,unit = 's')

In [11]:
#drop unwanted columns
df.drop(columns=['reviewTime','unixReviewTime','verified'],inplace=True)

In [12]:
df

,overall,reviewerID,asin,reviewText,summary,rev_date
0,5.0,A8LUWTIPU9CZB,0560467893,"Great product, love it!!",Five Stars,2015-11-05
1,3.0,A3B6GKQQ1JJ167,0560467893,"Pretty flimsy, but does the job. If your corne...",Meh,2015-05-07
2,5.0,A3MCTN65BU7XRA,0681795107,So much better than plastic mug types--keeps c...,Recommend,2014-01-22
3,1.0,A7JVZFSXVY9RL,0681795107,I like my coffee hot; borderline scorching but...,Not keeping coffee hot for long enough,2013-10-30
4,1.0,A2RQ7VLAK1SHPU,0681795107,This mug does only a fair job of keeping coffe...,Leaks like a waterfall when at an angle!,2013-09-20
...,...,...,...,...,...,...
6898950,5.0,AB4CZUDHN52H5,B01HJEJDBQ,I purchased these as decorations for my classr...,Beautiful Pom poms,2016-08-08
6898951,2.0,A12NA9GEGY6I3E,B01HJEOT2E,getting the used tea leaves out of this is ext...,cute but not very functional,2017-09-05
6898952,4.0,A38PQCNDGGWSPQ,B01HJEOT2E,Good deal. It took a while to recieve so I kin...,Good deal. It took a while to recieve so ...,2017-07-30
6898953,4.0,A2AY2C5EW8VOO7,B01HJGJNWS,Great price and okay quality. I know it's not ...,Four Stars,2018-07-28


In [13]:
#Sorting the rows by rev_date , old reviews to latest reviews
df = df.sort_values(by = 'rev_date')

In [14]:
#ADD CODE for finding duplicates
df[df.duplicated(subset = ['asin','reviewerID'], keep = 'last')]

,overall,reviewerID,asin,reviewText,summary,rev_date
21535,5.0,ALJGB7XHPFCR,B00004R92U,After mixing your dough and letting it rise th...,BETTER THAN SLICED BREAD!,2000-06-30
21690,5.0,ALJGB7XHPFCR,B00004R92U,After mixing your dough and letting it rise th...,A great bread-making tool,2000-07-11
43596,5.0,A231WM2Z2JL0U3,B00004SQ0K,"I've had mine for quite some time, and I would...",Perfect Results Always.,2000-09-05
27889,5.0,AX54G5AL870Q8,B00004RG94,Why do I like these so much? Because the othe...,My favorite set of measuring spoons.,2000-10-19
41002,4.0,AJF1X7RTIDD8Y,B00004SDTC,This toaster toasts just fine. The bread come...,"It's a fine toaster, but",2001-06-22
...,...,...,...,...,...,...
6788951,5.0,A30LCBJWW8EJXZ,B015MSI8WQ,This is a great gift for kids! It's Suitable f...,This is a great gift for kids,2018-09-01
5846814,5.0,A2NMV73BMOZCAR,B01CKM4DVC,They work great,Love them,2018-09-04
6871015,5.0,A20E61JYFUYE7B,B01EGRZB5Q,good,Five Stars,2018-09-05
6847758,3.0,A2SHJ3Y65BE7RT,B01C5JFOYU,I was a bit displeased with these as they're s...,Wouldnt buy again,2018-09-05


In [15]:
#Deleting these duplicate values keeping the latest review (the last value found)
df.drop_duplicates(subset = ['asin','reviewerID'], keep ='last', inplace = True)

In [16]:
df

,overall,reviewerID,asin,reviewText,summary,rev_date
20870,4.0,A2MNB77YGJ3CN0,B00004R940,I'll admit it -- I'm a bit of a snob when it c...,"I was skeptical, but now I'm sold . . .",2000-05-07
34742,5.0,A1LUUO72VAFKFJ,B00004S4TZ,Maybe when Mom cooked for a big family every d...,Superb. Why didn't somebody think of this yea...,2000-05-19
24851,5.0,ALSQSIHU2ETLM,B00004RDAZ,"This was an impulse buy, I was looking for a b...",What a wonderful kitchen appliance,2000-05-31
35097,2.0,A2E041EYGSY0HK,B00004S576,I bought this rice cooker based on the first t...,Really disappointed,2000-05-31
40953,5.0,A2RQOO8VYAEZZG,B00004SBIB,I went mad when I first saw this amazing cassa...,The lovliest Le Creuset ever!,2000-06-12
...,...,...,...,...,...,...
6897070,5.0,A2JROYIK9FSAL6,B01HCJCM52,These are exactly what I was looking for to so...,Perfect sheer curtains!,2018-10-03
6898692,5.0,A3A1GEBNMXSWQE,B01HIAZ9BY,Looks cheap but its sturdy to my surprise.,Great. Do recommend,2018-10-03
6898693,5.0,A3JE6DTQ3CZ2Q1,B01HIAZ9BY,Great look and quality,Great product,2018-10-03
6898453,5.0,A13KGVUSBEI7XG,B01HHAW9HW,This is a great lunchbox. My daughter loves th...,Its a good lunchbox.,2018-10-04


In [17]:
df.isna().sum()

overall          0
reviewerID       0
asin             0
reviewText    2376
summary       1205
rev_date         0
dtype: int64

In [18]:
#Check if both summary and review text are nan, delete those rows
cols = ['reviewText','summary']
df[df[cols].isnull().all(axis=1)]

,overall,reviewerID,asin,reviewText,summary,rev_date
3797106,5.0,A3DHCIDXXT0GO8,B00GCAHDJ8,NaN,NaN,2014-08-31
1137142,4.0,A2XX64MZUGGR35,B000XU49MW,NaN,NaN,2014-09-04
1677887,5.0,A3DHCIDXXT0GO8,B00240AIIS,NaN,NaN,2014-09-08
272221,3.0,A3SVGK8VCGYCOP,B0001MSDOC,NaN,NaN,2015-02-04
3209944,5.0,A3DHCIDXXT0GO8,B00AYTGPWE,NaN,NaN,2015-04-18
...,...,...,...,...,...,...
6716009,5.0,A1B6CFBNB9IE9C,B00UN0CMIA,NaN,NaN,2018-08-02
6815209,5.0,A2GVLM4T84L897,B018CLK436,NaN,NaN,2018-08-22
6767018,5.0,A2GVLM4T84L897,B012TXB572,NaN,NaN,2018-08-22
6820573,5.0,A2GVLM4T84L897,B0193K02LO,NaN,NaN,2018-08-22


In [19]:
idx = df[df[cols].isnull().all(axis=1)].index

In [20]:
df.drop(idx,inplace=True)

In [21]:
#Get all the indexes of reviewtext which has url
df.summary[df.summary.str.match(pat = '(https)|(http)|(www.)',na = False)]

1352090    http://www.amazon.com/gp/product/B0019S20CS/re...
1055867    http://www.amazon.com/gp/product/B0016NQ49Q/re...
100084     http://www.amazon.com/gp/product/B0000631ZM/re...
86087      http://www.amazon.com/gp/aw/d/B00005NCWS/ref=a...
3645618    http://www.amazon.com/gp/product/B00EUWISFS/re...
                                 ...                        
6892309                                https: //www. amazon.
4584161                                https: //www. amazon.
6025261                                https: //www. amazon.
6086577                                https: //www. amazon.
6517350                                https: //www. amazon.
Name: summary, Length: 235, dtype: object

In [22]:
#store the index
ids = df.summary[df.summary.str.match(pat = '(https)|(http)|(www.)',na = False)].index

In [23]:
#display a sample summary with url
df.summary[1352090]

'http://www.amazon.com/gp/product/B0019S20CS/ref=cm_cr_rev_prod_title'

In [24]:
#Replace url with ''
for i in ids:
    df.summary[i]=''

/home/014544907/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
#double check all summary if it still has url
id_s = df[df.summary.str.match(pat = '(https)|(http)|(www.)',na = False)].index

In [26]:
#Index of all the summary which has urls
id_s

Int64Index([], dtype='int64')

In [27]:
#combine both review and summary texts
df['reviewText'] = df[['reviewText', 'summary']].astype(str).apply(lambda x: ' //'.join(x), axis = 1)

In [28]:
#deleting summary column
del df['summary']

In [29]:
#reset index
df.reset_index(inplace=True,drop=True)

In [30]:
df

,overall,reviewerID,asin,reviewText,rev_date
0,4.0,A2MNB77YGJ3CN0,B00004R940,I'll admit it -- I'm a bit of a snob when it c...,2000-05-07
1,5.0,A1LUUO72VAFKFJ,B00004S4TZ,Maybe when Mom cooked for a big family every d...,2000-05-19
2,5.0,ALSQSIHU2ETLM,B00004RDAZ,"This was an impulse buy, I was looking for a b...",2000-05-31
3,2.0,A2E041EYGSY0HK,B00004S576,I bought this rice cooker based on the first t...,2000-05-31
4,5.0,A2RQOO8VYAEZZG,B00004SBIB,I went mad when I first saw this amazing cassa...,2000-06-12
...,...,...,...,...,...
6053291,5.0,A2JROYIK9FSAL6,B01HCJCM52,These are exactly what I was looking for to so...,2018-10-03
6053292,5.0,A3A1GEBNMXSWQE,B01HIAZ9BY,Looks cheap but its sturdy to my surprise. //G...,2018-10-03
6053293,5.0,A3JE6DTQ3CZ2Q1,B01HIAZ9BY,Great look and quality //Great product,2018-10-03
6053294,5.0,A13KGVUSBEI7XG,B01HHAW9HW,This is a great lunchbox. My daughter loves th...,2018-10-04


In [31]:
#code for removing url in any form
import re
def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http|www)(://|\.)(\w|\.|\/|\?|\=|\&|\%|[-]|\///)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

In [32]:
#Get all the indexes of reviewtext which has url
idrev = df[df.reviewText.str.match(pat = '(https)|(http)|(www.)')].index

In [33]:
#Index of all the reviews which has urls
idrev

Int64Index([  29728,   77664,   97556,  153047,  191329,  232195,  318518,
             361580, 1015598, 1018903,
            ...
            5794295, 5852077, 5867727, 5868000, 5897110, 5904426, 5918600,
            5918893, 5937074, 5996423],
           dtype='int64', length=265)

In [34]:
#Check a sample review test with url
df.reviewText[29728]

'http://www.amazon.com/SCG-5057-Bacon-Wave/dp/B0016P1AFM\n\nThe good news is that the bacon is literally grease-free, retains its shape and cooks to perfection.\n\nThe BAD news is that the composite material warped badly on the first use, despite instructions being followed. It is still usuable but this design and the materials needs to be improved. //Bacon Wave - the Good news and the Bad news'

In [35]:
#After removing url
remove_urls(df.reviewText[29728])

'\n\nThe good news is that the bacon is literally grease-free, retains its shape and cooks to perfection.\n\nThe BAD news is that the composite material warped badly on the first use, despite instructions being followed. It is still usuable but this design and the materials needs to be improved. //Bacon Wave - the Good news and the Bad news'

In [36]:
#Remove url for all the indexes which had url
for i in idrev:
    df.reviewText[i] = remove_urls(df.reviewText[i])

/home/014544907/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
#Double check if reviewtext still contains http/www
idrev = df[df.reviewText.str.match(pat = '(https)|(http)|(www.)')].index

In [38]:
#check index
idrev

Int64Index([], dtype='int64')

In [39]:
#Check for null values
df.isnull().sum()

overall       0
reviewerID    0
asin          0
reviewText    0
rev_date      0
dtype: int64

In [40]:
df

,overall,reviewerID,asin,reviewText,rev_date
0,4.0,A2MNB77YGJ3CN0,B00004R940,I'll admit it -- I'm a bit of a snob when it c...,2000-05-07
1,5.0,A1LUUO72VAFKFJ,B00004S4TZ,Maybe when Mom cooked for a big family every d...,2000-05-19
2,5.0,ALSQSIHU2ETLM,B00004RDAZ,"This was an impulse buy, I was looking for a b...",2000-05-31
3,2.0,A2E041EYGSY0HK,B00004S576,I bought this rice cooker based on the first t...,2000-05-31
4,5.0,A2RQOO8VYAEZZG,B00004SBIB,I went mad when I first saw this amazing cassa...,2000-06-12
...,...,...,...,...,...
6053291,5.0,A2JROYIK9FSAL6,B01HCJCM52,These are exactly what I was looking for to so...,2018-10-03
6053292,5.0,A3A1GEBNMXSWQE,B01HIAZ9BY,Looks cheap but its sturdy to my surprise. //G...,2018-10-03
6053293,5.0,A3JE6DTQ3CZ2Q1,B01HIAZ9BY,Great look and quality //Great product,2018-10-03
6053294,5.0,A13KGVUSBEI7XG,B01HHAW9HW,This is a great lunchbox. My daughter loves th...,2018-10-04


In [41]:
#Check if overall contains valid rating
df.overall.unique()

array([4., 5., 2., 1., 3.])

In [42]:
df[df.reviewText.str.contains(' NaN ')==True]

,overall,reviewerID,asin,reviewText,rev_date


In [43]:
df.to_pickle('Clean_data')